In [ ]:
import numpy as np 
import math 
from decimal import *
getcontext().prec = 10


## Dados de entrada

In [ ]:
#adicionar para os outros critérios de fadiga
d = ''
lim_esc = Decimal(800)
tensao_ruptura = 1200
p_max = 9000
montagem = 'fixação permanente'

aco = 'usinado_ou_laminado_frio'
fat_seg = Decimal(1.2)
conf = 50
temp = 80
carga = 'axial'
kp_linha = Decimal(2.8)

## Limite de resistência a fadiga (Cálculo dos K's)

### Acabamento superficial (Ka)

In [ ]:
def ka(aco):
  if aco == 'forjado':
    a = 272
    b = -0.995
    ka = a * (tensao_ruptura ** b)
  elif aco == 'laminado_quente':
      a = 57.7
      b = -0.718
      ka = a * (tensao_ruptura ** b)
  elif aco == 'usinado_ou_laminado_frio':
      a = 4.51
      b = -0.265
      ka = a * ((tensao_ruptura) ** b)
  elif aco == 'retificado':
      a = 1.58
      b = -0.085
      ka = a * (tensao_ruptura ** b)
  elif aco == 'espelhado':
      a = 1
      b = 0
      ka = a * (tensao_ruptura ** b)
  return ka

In [ ]:
ka = ka(aco)

### Tamanho da peça (Kb)

### Confiabilidade (Kc)

In [ ]:
def kc(conf):
  if conf == 50:
    kc = 1
    
  elif conf == 90:
      kc = 0.897

  elif conf == 95:
      kc = 0.868

  elif conf == 98:
      kc = 0.836

  elif conf == 99:
      kc = 0.814

  elif conf == 99.99:
      kc = 0.702

  elif conf == 99.9999:
      kc = 0.620

  elif conf == 99.99999:
      kc = 0.584

  return kc

In [ ]:
if kc(conf) > 1:
  kc = 1
else:
  kc = kc(conf)

In [ ]:
kc

1

### Temperatura (Kd)

In [ ]:
def kd(temp):
  kd = 0.9877 + (0.6507 * (10**-3) * temp) - (0.3414 * (10**-5) * (temp**2)) + (0.5621 * (10**-8) * (temp**3)) - (6.246 * (10**-12) * (temp**4))

  return kd

In [ ]:
if kd(temp) > 1:
  kd = 1
else:
  kd = kd(temp)

In [ ]:
kd

1

### Tipo de carga (Ke)

In [ ]:
def ke(carga):
  if carga == 'flexao':
    ke = 1
    
  elif carga == 'axial':
      ke = 0.85
      
  elif carga == 'torcao':
      ke = 0.577
      
  return ke

In [ ]:
  ke(carga)

0.85

### Fatores diversos (Kf)

In [ ]:
kf = 1

In [ ]:
#determinação dos limites de resistência para o cálculo das tensões
lim_normal_res = Decimal(ka* 1 * kc * kd * 0.85 * kf * 0.5 * tensao_ruptura)

print(lim_normal_res)

351.36919832857921619506669230759143829345703125


In [ ]:
#Cálculo da resistência de prova
rp = lim_esc * Decimal(0.855)

#Cálculo da Força Inicial
if montagem == 'fixação permanente':
    def fi(a):
      return (Decimal(0.9) * rp * a)
else:
    def fi(a):
      return (Decimal(0.75) * rp * a)

In [ ]:
def tensao_variavel(a):
  return (Decimal(1/9) * ( (kp_linha* p_max)/ (Decimal(2)*a)))

In [ ]:
def tensao_media(a):
  return tensao_variavel(a) + ((fi(a))/(a))

In [ ]:
#Determinação da função utilizada no método de Newton
def f(a):
  return 1 - (((fat_seg * tensao_variavel(a))/(lim_normal_res))**2)  - (((fat_seg * tensao_media(a))/(lim_esc))**2) 

In [ ]:
#Cálculo da derivada da função
def flin(f, a):  
  if a == 0:
    a = 1
    h = a / 5471
    flin = (1/(12*h)) * ( f(a- 2*h) - 8*f(a - h) + 8*f(a+h) - f(a + 2*h) )
  else:
    h = a / 5471
    flin = (1/(12*h)) * ( f(a- 2*h) - 8*f(a - h) + 8*f(a + h) - f(a + 2*h) )
  return flin


In [ ]:
#Cálculo do valor inicial da área
a0 = p_max /lim_esc
print(a0)

11.25


In [ ]:
#Método de newton
def newton(f, a0, epsilon, maxIter=20):
  if abs(f(a0)) <= epsilon:
    return (a0)

  print("k\t a0\t\t f(a0)") #cabeçalho da tabela que retornará com os valores embaixo. \t é o espaçamento que dou para ficar visual.
  
  k=1
  while k <= maxIter:
    a1 = a0 - f(a0)/flin(f, a0)
      
    print("%a\t%e\t%e" % (k, a1,f(a1)))

    if abs(f(a1)) <= epsilon:
      return a1      
    
    a0 = a1
    k = k + 1
  
  print('ERRO: Número máximo de iterações atingido')
  
  return a1

In [ ]:
#Resposta
raiz =  newton(f, a0, 10**-9, 20)

print(raiz)

k	 a0		 f(a0)
1	1.723813e+01	-1.694236e-01
2	2.438712e+01	-5.755132e-02
3	2.998466e+01	-1.234159e-02
4	3.193224e+01	-8.657618e-04
5	3.209027e+01	-4.917500e-06
6	3.209117e+01	-3.000000e-10
32.09117352
